In [7]:
# pip install opencv-python albumentations==1.4.0 numpy pandas
import cv2, os, glob, json, hashlib, numpy as np, pandas as pd
import albumentations as A

# กำหนด path ของโฟลเดอร์และไฟล์ที่จะใช้
SRC = r"C:\Users\flukt\AI\Template_final\DATA\Data_Cleaning\Crop"                     # ที่เก็บรูปต้นฉบับ (original images)
DST = r"C:\Users\flukt\AI\Template_final\DATA\Data_Cleaning\new"                     # ที่เก็บรูปที่ผ่านการ augment แล้ว
META_CSV = "dataset_change_image/meta/augment_meta.csv"  # ไฟล์ CSV สำหรับเก็บ metadata การแปลงรูป
os.makedirs(DST, exist_ok=True)# สร้างโฟลเดอร์สำหรับเก็บรูป augment ถ้ายังไม่มี
os.makedirs(os.path.dirname(META_CSV), exist_ok=True)# สร้างโฟลเดอร์สำหรับเก็บไฟล์ metadata (dataset/meta) ถ้ายังไม่มี ใช้ os.path.dirname เพื่อตัดชื่อไฟล์ออก เหลือแค่ path โฟลเดอร์

# ฟังก์ชันช่วยให้สุ่ม "คงที่ต่อไฟล์" (deterministic) เพื่อทำซ้ำได้
def seed_from_path(p):
    return int(hashlib.md5(p.encode("utf-8")).hexdigest()[:8], 16) % (2**31-1)

# ชุดทรานส์ฟอร์ม: เลือกแบบสุ่มจากพูล เพื่อให้ 1 รูป → ได้อย่างน้อย 12 แบบ
pool = [
    # A.Compose([A.LongestMaxSize(512), A.PadIfNeeded(512,512, border_mode=cv2.BORDER_CONSTANT, value=(114,114,114))]),
    # A.Compose([A.RandomResizedCrop(448, 448, scale=(0.7, 1.0), ratio=(0.9, 1.1))]),
    # A.Compose([A.HorizontalFlip(p=1.0)]),
    # A.Compose([A.VerticalFlip(p=1.0)]),
    A.Compose([A.Rotate(limit=25, border_mode=cv2.BORDER_REFLECT)]),
    # A.Compose([A.GaussianBlur(blur_limit=(3,7))]),
    # A.Compose([A.MotionBlur(blur_limit=7)]),
    # A.Compose([A.ImageCompression(quality_lower=35, quality_upper=65)]),
    # A.Compose([A.RandomBrightnessContrast(brightness_limit=0.25, contrast_limit=0.25)]),
    # A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),
    # A.Compose([A.Sharpen(alpha=(0.1,0.3), lightness=(0.7,1.0))]),
    # A.Compose([A.Perspective(scale=(0.05,0.08), keep_size=True)]),
    # A.Compose([A.CoarseDropout(max_holes=8, max_height=64, max_width=64, fill_value=(0,0,0))]),
    # A.Compose([
    # A.Downscale(scale_min=0.3, scale_max=0.6, interpolation=cv2.INTER_AREA),  # ย่อให้เนียน
    # A.Resize(448, 448, interpolation=cv2.INTER_CUBIC)]),                      # ขยายให้เนียน
]

records = []
num_per_image = 1  # >= 10 ตามโจทย์

for path in glob.glob(os.path.join(SRC, "*")):
    if not os.path.isfile(path): 
        continue
    img = cv2.imread(path)
    if img is None: 
        continue

    base = os.path.splitext(os.path.basename(path))[0]
    # แก้ EXIF orientation (กรณีรูปจากมือถือ) — ทางลัด: บังคับ rotate 0° ด้วย imdecode ก็ช่วยได้
    # ในตัวอย่างนี้ขอข้ามขั้นตอน EXIF ใน OpenCV (ถ้าต้องเคร่ง ให้ใช้ Pillow เปิดแล้วแปลงเป็น numpy)

    rng = np.random.RandomState(seed_from_path(path))
    choices = rng.choice(len(pool), size=num_per_image, replace=True)

    for i, idx in enumerate(choices):
        aug = pool[idx]
        augmented = aug(image=img)["image"]
        # บังคับขนาดผลลัพธ์ให้มาตรฐาน (เช่น 448x448)
        out = cv2.resize(augmented, (640,640), interpolation=cv2.INTER_AREA)

        out_name = f"{base}_aug_{i:02d}.jpg"
        out_path = os.path.join(DST, out_name)
        # ลดความชัดแบบควบคุมไฟล์ด้วย JPEG quality
        quality = int(rng.randint(40, 90))
        cv2.imwrite(out_path, out, [int(cv2.IMWRITE_JPEG_QUALITY), quality])

        records.append({
            "src": os.path.basename(path),
            "dst": out_name,
            "transform_index": int(idx),
            "transform_list": [t.__class__.__name__ for t in aug.transforms],
            "jpeg_quality": quality,
            "final_size": "448x448"
        })


pd.DataFrame(records).to_csv(META_CSV, index=False, encoding="utf-8-sig")
print(f"Done. Aug images -> {DST}, meta -> {META_CSV}, total aug: {len(records)}")


PermissionError: [Errno 13] Permission denied: 'dataset_change_image/meta/augment_meta.csv'